This notebook is used to reduce the dataset to only ten classes

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
path_to_image = "/content/gdrive/MyDrive/CUB_200_2011/images.txt"

In [ ]:
df = pd.read_csv(path_to_image, delimiter = " ", header=None, skiprows=-1)
#it seems that their ise some duplicates on the paths so we first delete it !
df = df.drop_duplicates(subset=1, keep='first')

In [ ]:
df

,0,1
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...
...,...,...
11783,11784,200.Common_Yellowthroat/Common_Yellowthroat_00...
11784,11785,200.Common_Yellowthroat/Common_Yellowthroat_00...
11785,11786,200.Common_Yellowthroat/Common_Yellowthroat_00...
11786,11787,200.Common_Yellowthroat/Common_Yellowthroat_00...


In [ ]:
def is_in_selected_classes(path_image):
  if int(path_image[:3])<=10:
    return True
  else:
    False

df["selected"] = df[1].apply(is_in_selected_classes)
df = df[df["selected"]==True]
del df["selected"]

In [ ]:
path_ = "/content/gdrive/MyDrive/CUB_200_2011/images2.txt"
df.to_csv(path_, header=None, index=None, sep=' ', mode='a')

In [ ]:
## Adapt target data for 10 classes

In [ ]:
path_to_target = "/content/gdrive/MyDrive/CUB_200_2011/image_class_labels.txt"

In [ ]:
df_target = pd.read_csv(path_to_target, delimiter = " ", header=None, skiprows=-1)

In [ ]:
df_target

,0,1
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
11783,11784,200
11784,11785,200
11785,11786,200
11786,11787,200


In [ ]:
results = pd.merge(
    df,
    df_target,
    how="left",
    on=0,)
   

In [ ]:
results

,0,1_x,1_y
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...,1
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...,1
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...,1
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...,1
...,...,...,...
538,539,010.Red_winged_Blackbird/Red_Winged_Blackbird_...,10
539,540,010.Red_winged_Blackbird/Red_Winged_Blackbird_...,10
540,541,010.Red_winged_Blackbird/Red_Winged_Blackbird_...,10
541,542,010.Red_winged_Blackbird/Red_Winged_Blackbird_...,10


In [ ]:
target =results["1_y"]

In [ ]:
target = target.to_numpy()
target = np.array([[ele] for ele in target])

In [ ]:
#encode one hot encoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(target)


OneHotEncoder(handle_unknown='ignore')

In [ ]:
target = enc.transform(target)

In [ ]:
target = target.toarray()

In [ ]:
target.shape

(543, 10)

In [ ]:
##save target 
np.savetxt('/content/gdrive/MyDrive/CUB_200_2011/target_reduced.txt', target, fmt='%d')

Same Work with text descriptions


In [ ]:
path_to_attributes = "/content/gdrive/MyDrive/CUB_200_2011/attributes.txt"
attributes = pd.read_csv(path_to_attributes, delimiter = " ", header=None)# ,nrows=nrows)
attributes.columns=["id_attribute","attributes"]

## The process to read the file is particular because it seems that some of the data is corrupted
path_to_image_attributes = "/content/gdrive/MyDrive/CUB_200_2011/attributes/image_attribute_labels.txt"

### Loop the data lines
with open(path_to_image_attributes, 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split(" ")) for l in temp_f.readlines() ]

### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
column_names = [i for i in range(0, max(col_count))]

### Read csv
image_attributes = pd.read_csv(path_to_image_attributes, header=None, delimiter=" ", names=column_names)#, nrows=nrows)
image_attributes = image_attributes[[0,1,2]]
image_attributes.columns = ["image_id", "id_attribute", "is_present",]

#we consider only rows where attribute is present
image_attributes = image_attributes[image_attributes["is_present"]==1]

result_att = pd.merge(image_attributes, attributes, on="id_attribute")
result_att = result.sort_values(by=['image_id'])

result_att['attributes'] = result.groupby(['image_id'])['attributes'].transform(lambda x: ' '.join(x))
result_att = result[['image_id','attributes']].drop_duplicates()
result_att["attributes"] = result["attributes"].str.replace("::"," ")

result_att

,image_id,attributes
0,1,has_bill_shape hooked_seabird has_eye_color br...
38363,2,has_breast_color white has_upperparts_color gr...
104704,3,has_wing_color buff has_eye_color black has_be...
96475,4,has_wing_color brown has_upperparts_color brow...
96476,5,has_wing_color brown has_breast_pattern solid ...
...,...,...
1255,11784,has_head_pattern masked has_bill_length shorte...
343133,11785,has_throat_color yellow has_head_pattern maske...
299704,11786,has_bill_shape all-purpose has_shape perching-...
309743,11787,has_under_tail_color buff has_primary_color bu...


In [ ]:
id_saved = list(results[0])
def saved(id_):
  if id_ in id_saved:
    return True 
  else: return False

In [ ]:
result_att = result_att[result_att["image_id"].apply(saved) == True]
result_att.reset_index()
result_att = result_att[["image_id", "attributes"]]
#del result_att["index"]

In [ ]:
result_att.to_csv("/content/gdrive/MyDrive/CUB_200_2011/text_attributes_reduced.csv")